<div style='text-align: right'><b> Ana Valentina López Chacón </b></div>
<div style='text-align: right'><b> Enero, 2026 </b></div>

# **Prueba Tecnica Desarrollador IA**

In [ ]:
!pip install openai pydantic langchain langchain-core pypdf --quiet

In [47]:
from typing import List
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
import os
from openai import AzureOpenAI
import httpx
from apikey import subscription_key, api_version, endpoint

In [48]:

model_name = "gpt-4o-mini"
deployment = "doc-classifier-gpt4o-mini"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

Definición del prompt y estructura de la salida usando Pydantic

In [49]:
class ClassificationResponse(BaseModel):
    category: str = Field(
        ..., 
        description="One of: Contrato, Queja o Reclamo, Resolución, Informe técnico, Comunicación interna"
    )
    justification: str = Field(
        ..., 
        description="2–3 sentence explanation citing textual evidence"
    )

parser = PydanticOutputParser(pydantic_object=ClassificationResponse)

def classify_document(text: str) -> ClassificationResponse:
    format_instructions = parser.get_format_instructions()

    prompt = PromptTemplate(
        template=(
            "You are an AI system designed to classify administrative and technical documents.\n\n"
            "Classify the document into EXACTLY ONE of the following categories:\n"
            "1. Contrato\n"
            "2. Queja o Reclamo\n"
            "3. Resolución\n"
            "4. Informe técnico\n"
            "5. Comunicación interna\n\n"
            "Definitions:\n"
            "- Contrato: Legal or formal agreements defining obligations, duration, and terms.\n"
            "- Queja o Reclamo: Expressions of dissatisfaction, complaints, or requests for correction.\n"
            "- Resolución: Official decisions issued by an authority that resolve a matter.\n"
            "- Informe técnico: Technical or analytical documents presenting results, evaluations, or recommendations.\n"
            "- Comunicación interna: Internal memos, emails, or notices intended for internal organizational use.\n\n"
            "Respond ONLY with a JSON object following this format:\n"
            "{format_instructions}\n\n"
            "Document:\n"
            "{text}"
        ),
        input_variables=["text"],
        partial_variables={"format_instructions": format_instructions},
    )

    final_prompt = prompt.format(text=text)

    response = client.chat.completions.create(
        model = deployment, 
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a precise document classification system. "
                    "Follow the instructions strictly. "
                    "Do not add any extra fields."
                )
            },
            {
                "role": "user",
                "content": final_prompt
            }
        ],
        temperature=0,
        max_tokens=500
    )

    raw_output = response.choices[0].message.content

    return parser.parse(raw_output)

## **Leer y limpiar archivos PDF**

In [50]:
from pypdf import PdfReader
import re
from collections import defaultdict

In [51]:
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    pages_text = []

    for i, page in enumerate(reader.pages):
        page_text = page.extract_text()
        if page_text:
            pages_text.append(page_text)

    full_text = "\n".join(pages_text)
    return full_text

def clean_text(text):
    text = text.replace("\r", "\n")
    text = re.sub(r"\n{3,}", "\n\n", text)
    text = re.sub(r"[ \t]{2,}", " ", text)
    text = text.strip()
    return text

def truncate_text(text, max_chars = 12000):
    if len(text) <= max_chars:
        return text
    return text[:max_chars]


In [52]:
PII_PATTERNS = {
    "EMAIL": r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+",
    "PHONE": r"\b(\+?\d{1,3})?\s?\d{3}\s?\d{3}\s?\d{4}\b",
    "ID": r"\b(CC|TI|NIT)\s?\d{6,12}\b",
    "PERSON": r"\b[A-ZÁÉÍÓÚÑ][a-záéíóúñ]+(?:\s[A-ZÁÉÍÓÚÑ][a-záéíóúñ]+)+\b"
}


def pseudonymize_text(text):
    counters = defaultdict(int)
    entity_map = {}

    def replace(entity_type, match):
        value = match.group(0)
        if value not in entity_map:
            counters[entity_type] += 1
            entity_map[value] = f"[{entity_type}_{counters[entity_type]}]"
        return entity_map[value]

    for entity_type, pattern in PII_PATTERNS.items():
        text = re.sub(pattern, lambda m: replace(entity_type, m), text)

    return text, entity_map

In [55]:
pdf_path = "./Ejemplos/comunicacion.pdf"

raw_text = extract_text_from_pdf(pdf_path)
cleaned_text = clean_text(raw_text)
truncated_text = truncate_text(cleaned_text)

masked_text, pii_map = pseudonymize_text(truncated_text)
masked_text

'Para: Equipo de Operaciones \nDe: Dirección de Tecnología \nAsunto: Lineamientos para el manejo de información sensible \nFecha: [fecha] \nPor medio de la presente se informa a todos los miembros del equipo de operaciones que, \na partir de la fecha, se deberán seguir nuevos lineamientos para el manejo y tratamiento de \ninformación sensible dentro de los proyectos en curso. \nEs obligatorio asegurar que los documentos que contengan datos personales sean \ntratados conforme a las políticas internas de seguridad de la información, evitando su \ndivulgación no autorizada y garantizando su adecuado almacenamiento. \nAdicionalmente, se solicita a cada líder de equipo socializar estas directrices con sus \nrespectivos grupos de trabajo y verificar su correcta implementación. \nPara cualquier duda adicional, pueden comunicarse con la Dirección de Tecnología. \nCordialmente, \n[Nombre del Responsable] \nDirección de Tecnología'

In [56]:
result = classify_document(masked_text)
print(result.category)
print(result.justification)

Comunicación interna
The document is an internal memo from the Dirección de Tecnología to the Equipo de Operaciones, outlining new guidelines for handling sensitive information. It is intended for internal organizational use and includes instructions for compliance and communication within the team.
